<a href="https://colab.research.google.com/github/patelmeshwa99/Sentiment-Analysis/blob/master/on_sentiment140_countVector_%2B_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

--2020-06-20 12:18:48--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  82.8MB/s    in 1.0s    

2020-06-20 12:18:49 (82.8 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  


In [2]:
import pandas as pd

df = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
df.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.polarity = df.polarity.replace({0: 0, 4: 1})
df.polarity.value_counts()

1    800000
0    800000
Name: polarity, dtype: int64

In [4]:
df = df.drop(columns=['id', 'date', 'query', 'user'])
df.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df = df.sample(n=25000)
df.to_csv("data/sentiment140-subset.csv", index=False)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(
     df, test_size=0.2, random_state=4)

In [7]:
train.head()

,polarity,text
904499,1,@annahasphyxiate re sore back and legs: well a...
91300,0,"another work day, another bored day"
847675,1,Extremely sleepy. Wisdom teeth bothering me. T...
1324940,1,@witchyknitter I've commented twice and nothin...
344587,0,@luke uurrrghh! Rough isn't it? You guys get ...


In [8]:
sum(train["polarity"] == 0)

10139

In [9]:
sum(train["polarity"] == 1)

9861

In [10]:
train.isnull().sum()


polarity    0
text        0
dtype: int64

In [11]:
!pip install tweet-preprocessor


In [12]:
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [13]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [14]:
train_tweet = clean_tweets(train["text"])


In [15]:
train["text"] = train_tweet

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
train.head()

,polarity,text
904499,1,re sore back and legs well at least you had a ...
91300,0,another work day another bored day
847675,1,extremely sleepy wisdom teeth bothering me tum...
1324940,1,ive commented twice and nothing let me know if...
344587,0,uurrrghh rough isnt it you guys get mags full ...


In [17]:
# train["clean_tweet"] = train_tweet

# # compare the cleaned and uncleaned tweets
# train.head(10)

In [18]:
sum(train["polarity"] == 0)

10139

In [19]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["text"])
test["text"] = test_tweet
# append cleaned tweets to the training data
# test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,polarity,text
273970,0,ugh monday
1367862,1,looking forward to it
761704,0,that sucks but its ok cause everybodys got a w...
267859,0,folding my clothes so boring
240702,0,some of the best shots of my life i cant show ...


In [20]:
sum(train["polarity"]==1)

9861

In [21]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.polarity.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.text.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer


In [23]:
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [24]:
sum(train.polarity == 1)

9861

In [25]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [26]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  70.41666666666667 %
